## Socket

官方文档阅读笔记

---

该模块支持访问 `BSD Socket` 套接字，跨操作系统平台可以使用，但是有些操作是操作系统平台相关的，因为有时候会访问操作系统的 Socket API 接口

函数 `socket()` 创建了套接字对象，并且该对象提供了大量的支持各种套接字系统调用的方法,提供了比 C 高级的参数类型接口，正如在 Python 中 `read(), write()` 函数操作文件那样，接收操作的缓冲区内存分配是自动的并且缓冲区长度隐含在发送操作中	

---
1. CS架构
    
    ![](./CS.png)

2. 地址族
    
    ![](./AF.png)
    ```python
    import socket
    socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)    # 前一个参数固定，后一个参数表示采用流式 TCP 链接
    socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)     # 前一个参数固定，够一个参数表示采用数据报式的 UDP 链接
    ```

3. 套接字族
    1. 支持多种套接字的格式和构建
    2. 对于 `AF_INET` 地址族来说，需要使用 `(host, port)` 元组构建
    3. 对于 `AF_INET6` 地址族来说，需要使用 `(host, port, flowinfo, scopeid)` 元组构建
    

---
1. 模块组织
    * 异常模块 : `OSError` 的子类
    * 常量 (`Int` 类型)
    
        `socket.AF_UNIX`  
        `socket.AF_INET`  
        `socket.AF_INET6`
        
        地址(协议)族用于构建 `socket()` 对象的第一个参数
        
        `socket.SOCK_STREAM`: `TCP`  
        `socket.SOCK_DGRAM`: `UDP`  
        `socket.SOCK_RAW`: 原始套接字  
        `socket.SOCK_RDM`  
        `socket.SOCK_SEQPACKET`
        
        表示套接字的类型，用于构建 `socket()` 对象的第二个参数
        
    * 功能函数
        1. 构建套接字
            * ```socket.socket(family=AF_INET, type=SOCK_STREAM, proto=0, fileno=None)```
                1. 默认是网络 TCP 套接字
                2. 使用给定的地址族，套接字类型，协议号等构建套接字对象
                3. 协议号一般都是 0
                4. `fileno` 参数制定的话，其他的参数都是无效的，表示使用特定的文件描述符构建套接字
                5. 支持使用上下文管理器 `with`
        2. 其他函数
            * ```socket.gethostbyname(hostname)```: 通过网站的主机名获取对应的 `IP` 地址,只支持 `IPv4`,返回字符串
                
                ```python
                socket.gethostbyname('www.google.com')
                ```
                
            * ```socket.gethostname()``` : 返回当前的 `Python` 解释器存在的主机名
                ```python
                socket.gethostname()    # GMFTBY
                ```
                
            * ```socket.getdefaulttimeout()``` : 返回当前的套接字的超时限制时间(s),`None` 表示不会超时

            * ```socket.setdefaulttimeout(timeout)``` : 设置超时时间(s)  

        3. `Socket` 对象函数
            * ```socket.accept()```  
                接收连接的函数, `socket` 对象必须要保证已经绑定到对应的端口和主机上并且开始监听，返回一个元组 `(conn, address)`
                其中的 `conn` 表示可以和链接的套接字发送和接收数据的套接字, `address` 表示连接到的套接字的地址
                服务器读取和发送的数据一般都是对 `accept` 返回的新的套接字进行读取和写入操作
            * `socket.bind(address)`  
                绑定 `socket` 套接字到对应的主机和端口上，绑定的方式取决于套接字的类型，并且不能之前被绑定过
            * `socket.close()`  
                关闭原语表示无效化一个 `socket` 之后在这个套接字上的操作都会失败，垃圾回收器会自动关闭但是最好还是显示的关闭防止意外的发生
                或者可以考虑使用 `with` 结构使用上下文管理器，但是 `close()` 函数只是释放资源，并不是及时的关闭，可以考虑其他的函数强制关闭
            * `socket.connect(address)`   
                连接地址上标注的远程套接字
            * `socket.getpeername()` : 返回远程连接的套接字的端口和地址，连接之后才可以查看结果
            * `socket.getsockname()` : 返回本地套接字的端口和地址,`socket.bind` 可以查看结果
            * `socket.gettimeout()`
            * `socket.listen(int)`:
                使服务器可以接受连接，一般是5即可
            * `socket.recv(bufsize[, flags])`  
                从套接字接收数据,返回 `bytes` 其中的 `bytes` 表示从远程套接字中接收到的字节型数据,阻塞等待知道接收到一个数据或者是连接关闭，`bufsize` 一般最好是2的指数表示一次最多可以接收的数据的数目
                **如果断开连接的话，调用该函数可能会立即的获得 `b''` 这样的空字符串，这种情况下，我们为了保证正确需要将该 `socket` 关闭 `socket.close()`, 这一点在 `select` 的多路 I/O 复用中显得尤为重要**
            * `socket.recvrfom(bufsize[, flags])`
                类似的，但是返回 `(bytes, address)` 还返回发送者的地址
            * `socket.send(bytes[, flags])`:  
                **注意发送的数据必须是字节类型的**,发送数据,返回发送的数据的字节大小,发送TCP数据，返回发送的字节大小。这个字节长度可能少于实际要发送的数据的长度。换句话说，这个函数执行一次，并不一定能发送完给定的数据，**可能需要重复多次才能发送完成。**
            * `socket.sendall(bytesp, flags[)`: 类似 `send` ，发送完整的TCP数据，成功返回None，失败抛出异常,保证了一次传输可能定会完全传输
            * `socket.sendto(bytes, address)` : 
                主要用在 `UDP` 发送地址上, `UDP` 连接并不需要一个完整的连接，只需要制定出发送地址即可，`bytes` 是发送数据， `address` 是根据地址族的地址(地址和端口号)，返回发送的字节数据大小
            
2. 套接字 `timeout` 注意点
    1. 套接字的状态 : 阻塞，不阻塞，限制时长 `timeout`, **创建的套接字默认都是阻塞模式**，通过修改 `setdefaulttimeout` 可以修改这个选项
    2. 阻塞 : 
        在阻塞的情况下，程序会一直阻塞到异常(超时 `timeout` 等等)或者完成操作
    3. 非阻塞模式 :
        不能立即完成的操作会失败
    4. 超时模式下 : 
        如果不能在制定的时间内 `timeout` 完成操作就会发生失败，或者是系统发生异常操作失败
    5. `connect` 操作也是需要进行超时处理的(最好)，一般在 `connect` 之前需要使用功能 `settimeout` 函数设定超时的时间
    
3. 实例模拟  
---
`TCP`
    * 服务器的基本流程
        1. `socket()`
        2. `bind()`
        3. `listen()`
        4. `accept()` : 当然这个操作可以是多次循环的，以便帮助服务器服务多个客户端
    * 客户端基本流程
        1. `socket()`
        2. `connect()`  
 
---
`UDP`  
    * 服务器流程
        1. `socket()`
        2. `bind()`
    * 客户端流程
        1. `socket()`

### 个人记录
---
1. 检测当前的 `socket` 的连接状态
    * 对于 `recv`, `send` 函数的异常可以判断当前的交流管道是不是存在有异常或者说是断开

In [1]:
# 支持 IPv4 的服务端，使用上下文管理器
import socket

HOST = ''                 # 注意如果出现了 [111] 错误的话，一般将 HOST 改成 '' 就可以实现
PORT = 50007              # Arbitrary non-privileged port
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen(1)
    conn, addr = s.accept()
    # conn 是返回的和远程的套接字连接的套接字，也可以使用 with 上下文管理器
    with conn:
        print('Connected by', addr)    # 打印连接地址
        while True:
            data = conn.recv(1024)
            if not data: break
            conn.sendall(data)

Connected by ('127.0.0.1', 36752)


In [ ]:
# 上述服务端对应的客户端,只支持 IPv4
HOST = '127.0.0.1'    # The remote host
PORT = 50007              # The same port as used by the server
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(b'Hello, world')
    data = s.recv(1024)
print('Received', repr(data))